In [4]:
# chargement des extensions
%reload_ext autoreload
%autoreload 2

# import des librairies
from MODULES.loader import IntermovieDataLoader
import pandas as pd

In [44]:
# Supprimer à l'import
title_ratings = pd.read_csv("../datas/movies_dataset/title.ratings.tsv", sep='\t', index_col='tconst', encoding='utf-8')
title_akas = pd.read_csv("../datas/movies_dataset/title.akas.tsv", sep='\t', index_col='titleId', usecols=['titleId', 'region'], encoding='utf-8')
title_akas = pd.DataFrame(index=title_akas.index.drop_duplicates())

FileNotFoundError: [Errno 2] No such file or directory: '../movies_dataset/title.akas.tsv'

In [22]:
nb_votes = 400

data_loader = IntermovieDataLoader()
data_loader.split_data_role()
data_loader.split_data_type()

In [29]:
actors = pd.concat([pd.read_csv('../datas/CURATED/actor.csv', delimiter=',', usecols=['tconst', 'nconst'], index_col = ['tconst']), pd.read_csv('../datas/CURATED/actress.csv', delimiter=',', usecols=['tconst', 'nconst'], index_col = ['tconst'])])
actors = actors.rename(columns={"nconst": "actors"})

movies = pd.read_csv('../datas/CURATED/movie.csv', delimiter=',', usecols=['tconst', 'isAdult', 'startYear', 'runtimeMinutes', 'genres'], index_col = ['tconst'])
movies = movies[movies.runtimeMinutes != '\\N'][movies.genres != '\\N']

dataset = actors.merge(movies, how='right', left_index=True, right_index=True)
dataset = dataset.dropna()
dataset = dataset.assign(genres=dataset['genres'].str.split(',')).explode('genres')

In [42]:
producers = pd.read_csv('../datas/CURATED/producer.csv', delimiter=',', usecols=['tconst', 'nconst'], index_col = ['tconst'])
producers = producers.rename(columns={"nconst": "producer"})

writers = pd.read_csv('../datas/CURATED/writer.csv', delimiter=',', usecols=['tconst', 'nconst'], index_col = ['tconst'])
writers = writers.rename(columns={"nconst": "writer"})

composers = pd.read_csv('../datas/CURATED/composer.csv', delimiter=',', usecols=['tconst', 'nconst'], index_col = ['tconst'])
composers = composers.rename(columns={"nconst": "composer"})

producers = producers.merge(writers, how="inner", left_index=True, right_index=True)
producers = producers.merge(composers, how="inner", left_index=True, right_index=True)
dataset = dataset.merge(producers, how="left", left_index=True, right_index=True)
dataset = dataset.dropna()

rated_movies = title_ratings[title_ratings.numVotes  > nb_votes]
rated_movies = rated_movies[['averageRating']]
dataset = dataset.merge(rated_movies, how='left', left_index=True, right_index=True)
dataset = dataset.dropna()